In [1]:
import sys
sys.path.append('/mnt/')
import numpy as np
import pandas as pd
from finrl.models.env import StockPortfolioEnv
from finrl.models.utils import data_split
from finrl.models.metrics import *
from finrl.models.constants import *
from finrl.data.generate import portfolio_data_generate  # 数据生成
from finrl.models.baselines.statistics import *
# import finrl.models.baselines.statistics as statistics   # 将statistics当作包导入，前提是具有init文件。
# statistics.BestCRP  # init文件下导入的包

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**注意** :

不同的python版本、不同的python包(如numpy、scipy等)的版本、不同的cpu都可能导致计算精度的差异，从而造成评估结果和论文有甚至较大的出入。

In [ ]:
data_dir = '/mnt/finrl/data/csv/DOW_30/'
vix_data_dir = '/mnt/finrl/data/csv/^VIX/'
dji_dir = '/mnt/finrl/data/csv/DJI/DJI.csv'  # .csv

TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-12-31'
VALIDATE_START_DATE = '2022-01-01'
VALIDATE_END_DATE = '2022-12-31'
TEST_START_DATE = '2023-01-01'
TEST_END_DATE = '2024-01-30'

df = portfolio_data_generate(
    data_dir=data_dir,
    start_date=TRAIN_START_DATE,
    end_date=TEST_END_DATE,
    use_technical_indicator=True,
    use_turbulence=True,
    user_defined_feature=False,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    vix_data_dir=vix_data_dir,
    dji_dir=dji_dir,
)

df_train = data_split(df,TRAIN_START_DATE,TRAIN_END_DATE)
df_validation = data_split(df,VALIDATE_START_DATE,VALIDATE_END_DATE)
df_test = data_split(df,TEST_START_DATE,TEST_END_DATE)
df_train_validation = data_split(df,TRAIN_START_DATE,VALIDATE_END_DATE)


In [ ]:
env_portfolio_kwargs = {
    'stock_dim': None,    # len(df.tic.unique())
    'hmax': 100,
    'initial_amount': 1000000,
    'buy_cost_pct': None,
    'sell_cost_pct': None,
    'reward_scaling': 1e-4,
    'tech_indicator_list': ['macd', 'rsi_30', 'cci_30', 'dx_30'],
}

In [ ]:
ACTION_DIM = len(df_train.tic.unique())
STATE_DIM = ACTION_DIM + 4  # 理解为每只股票的状态个数，二维状态矩阵的行数

stock_dim = len(df.tic.unique())
env_portfolio_kwargs.update({
                            'stock_dim':stock_dim,
                            'buy_cost_pct':[0.001]*ACTION_DIM,
                            'sell_cost_pct':[0.001]*ACTION_DIM,
                            })
env_test = StockPortfolioEnv(df_test, **env_portfolio_kwargs)
print(env_portfolio_kwargs)

# Best CRP

In [5]:
bcrp = BestCRP(df_train_validation,df_test)

In [22]:
s = env_test.reset()  # list
done = False
while not done:
    a = bcrp.decide().reshape(-1)  # (action_dim,)
    s_, r, done, _ = env_test.step(a)  # ndarray,float,bool,dict
    s = s_

In [23]:
print(f'Sharpe Ratio: {sharpe_ratio(env_test.returns)}')
print(f'Cumulative Return: {cum_returns_final(env_test.returns)}')
print(f'Annual Return: {annual_return(env_test.returns)}')
print(f'Max Drawdown{max_drawdown(env_test.returns)}')

Sharpe Ratio: 1.4303
Cumulative Return: 0.1766
Annual Return: 0.1646
Max Drawdown-0.0831


# BestStock

In [19]:
bs = BestStock(df_train_validation, df_test)
s = env_test.reset()  
done = False
while not done:  
    a = np.zeros([29,1]).reshape(-1,)
    a[24]=1
    # a = bs.decide().reshape(-1)  
    s_, r, done, _ = env_test.step(a)  
    s = s_

In [20]:
print(f'Sharpe Ratio: {sharpe_ratio(env_test.returns)}')
print(f'Cumulative Return: {cum_returns_final(env_test.returns)}')
print(f'Annual Return: {annual_return(env_test.returns)}')
print(f'Max Drawdown{max_drawdown(env_test.returns)}')

Sharpe Ratio: 1.3952
Cumulative Return: 0.1703
Annual Return: 0.1587
Max Drawdown-0.0804


# UBAH

In [26]:
ubah = UniformBAH(df_train_validation, df_test)

s = env_test.reset()  
done = False
while not done:
    
    last_weights = env_test.last_weights
    a = ubah.decide(last_weights).reshape(-1)  
    s_, r, done, _ = env_test.step(a)  
    s = s_

In [27]:
print(f'Sharpe Ratio: {sharpe_ratio(env_test.returns)}')
print(f'Cumulative Return: {cum_returns_final(env_test.returns)}')
print(f'Annual Return: {annual_return(env_test.returns)}')
print(f'Max Drawdown{max_drawdown(env_test.returns)}')

Sharpe Ratio: 1.4343
Cumulative Return: 0.1808
Annual Return: 0.1685
Max Drawdown-0.0872


# Uniform CRP

In [35]:
ucrp = UniformCRP(df_train_validation, df_test)

s = env_test.reset()  
done = False
while not done:
    a = ucrp.decide().reshape(-1)  
    s_, r, done, _ = env_test.step(a)  
    s = s_

In [38]:
print(f'Sharpe Ratio: {sharpe_ratio(env_test.returns)}')
print(f'Cumulative Return: {cum_returns_final(env_test.returns)}')
print(f'Annual Return: {annual_return(env_test.returns)}')
print(f'Max Drawdown{max_drawdown(env_test.returns)}')

Sharpe Ratio: 1.4334
Cumulative Return: 0.1807
Annual Return: 0.1684
Max Drawdown-0.0872


In [37]:
with open('ucrp_assets.txt','w') as f:
    for asset in env_test.asset_memory:
        f.write(str(asset)+'\n')

# Universal Portfolio


In [39]:
up = UniversalPortfolio(df_train_validation, df_test)
s = env_test.reset()
done = False
while not done:
    a = up.decide().reshape(-1)  
    s_, r, done, _ = env_test.step(a)  
    s = s_

In [40]:
print(f'Sharpe Ratio: {sharpe_ratio(env_test.returns)}')
print(f'Cumulative Return: {cum_returns_final(env_test.returns)}')
print(f'Annual Return: {annual_return(env_test.returns)}')
print(f'Max Drawdown: {max_drawdown(env_test.returns)}')

Sharpe Ratio: 1.4549
Cumulative Return: 0.1823
Annual Return: 0.1699
Max Drawdown: -0.0751
